In [1]:
import ctypes
import gc

In [2]:
# 工具函数1：通过内存地址查引用计数
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [3]:
# 工具函数2：通过内存地址查对象是否在垃圾回收跟踪列表
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not Found"

In [4]:
# 定义类A和B，互相引用形成循环
class A:
    def __init__(self):
        self.b = B(self)  # A的属性指向B，传入自身
        print(f'A: self: {hex(id(self))}, b: {hex(id(self.b))}')

class B:
    def __init__(self, a):
        self.a = a  # B的属性指向A
        print(f'B: self: {hex(id(self))}, a: {hex(id(self.a))}')

# 步骤 1：创建循环引用对象

先关闭垃圾回收器（方便观察），然后创建`A`的实例`my_var`：

In [6]:
gc.disable()  # 关闭垃圾回收器
my_var = A()

B: self: 0x104772490, a: 0x104772790
A: self: 0x104772790, b: 0x104772490


# 步骤 2：查看初始引用计数

我们先记录两个对象的内存地址，再查引用数：

In [7]:
# 记录A和B的内存地址
a_id = id(my_var)  # A的地址
b_id = id(my_var.b)  # B的地址

# 查引用计数
print(ref_count(a_id))  # 输出：2
print(ref_count(b_id))  # 输出：1

2
1


# 步骤 3：销毁外部引用，循环引用 “显形”

把`my_var`指向`None`，销毁外部引用：

In [8]:
my_var = None

# 再查引用计数
print(ref_count(a_id))  # 输出：1
print(ref_count(b_id))  # 输出：1

1
1


# 二、垃圾回收器：专门收拾循环引用的 “清洁工”

我们手动触发一次垃圾回收，看看效果：

In [9]:
# 手动触发垃圾回收
gc.collect()

# 检查对象是否存在
print(object_by_id(a_id))  # 输出：Not Found
print(object_by_id(b_id))  # 输出：Not Found

Not Found
Not Found


In [18]:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            print(f"当前占用该地址的对象类型：{type(obj)}")  # 新增：打印对象类型
            return "Object exists"
    return "Not Found"

# 执行gc.collect()后调用
object_by_id(a_id)

当前占用该地址的对象类型：<class 'cell'>


'Object exists'

# 三、垃圾回收器的关键知识点

1. **默认开启**：Python 默认会自动运行 GC，不用手动干预；
2. **手动触发**：用`gc.collect()`可以强制回收，适合需要精准控制内存的场景；
3. **关闭需谨慎**：`gc.disable()`能关闭 GC，但如果代码有循环引用，必然会内存泄漏；
4. **版本兼容**：Python 3.4 之前，带`__del__`析构函数的循环引用对象无法被 GC 回收，但 3.4 + 已经修复了这个问题。

## 总结

- 引用计数是内存管理的基础，但搞不定**循环引用**；
- 垃圾回收器通过**可达性分析**，专门回收循环引用产生的 “不可达对象”；
- 用代码实战能直观看到：销毁外部引用后，循环引用对象的引用数不为 0，但 GC 能识别并回收它们。